In [1]:
import pandas as pd
import City_Preprocessing as bkPro

city_raw = pd.read_csv('/Users/briankalinowski/PycharmProjects/CIS600/zecon/City_time_series.csv')

In [11]:

# Should ALWAYS have Date and RegionName
features = ['Date', 'RegionName',
                'PriceToRentRatio_AllHomes',
                'ZRI_AllHomes',
                'ZriPerSqft_AllHomes',
                'Zri_SingleFamilyResidenceRental']

# all our measurement Featreus
values = ['PriceToRentRatio_AllHomes',
                'ZRI_AllHomes',
                'ZriPerSqft_AllHomes',
                'Zri_SingleFamilyResidenceRental']

lookup_ix = ['State', 'City']
full_ix = ['State', 'City', 'Date']

gentrified = bkPro.clean_city_data(city_raw, features, values)
gentrified = bkPro.merge_location_data(gentrified)

In [12]:
gentrified = bkPro.set_city_indices(gentrified, full_ix)

In [13]:
gentrified.head()

PriceToRentRatio_AllHomes  ZRI_AllHomes  \
State City         Date                                                  
AK    Anchor Point 2011-01-31                      20.80           NaN   
                   2011-02-28                      21.39           NaN   
                   2011-03-31                      17.67        1363.0   
                   2011-04-30                      19.16        1344.0   
                   2011-05-31                      19.04        1325.0   

                               ZriPerSqft_AllHomes  \
State City         Date                              
AK    Anchor Point 2011-01-31                  NaN   
                   2011-02-28                  NaN   
                   2011-03-31                1.090   
                   2011-04-30                1.068   
                   2011-05-31                1.048   

                               Zri_SingleFamilyResidenceRental  
State City         Date                                         
AK    Anchor Point 2011-01-31                              NaN  
                   2011-02-28                              NaN  
                   2011-03-31                           1354.0  
                   2011-04-30                           1330.0  
                   2011-05-31                           1307.0

In [14]:
gent = [('NC', 'Asheville'), ('TN', 'Nashville'), ('CA', 'Oakland'), ('SC', 'Charleston'), ('CA', 'Anaheim'),
            ('CA', 'Berkeley'), ('WA', 'Seattle'), ('TX', 'Austin'), ('CA', 'Los Angeles'), ('CA', 'San Diego'),
            ('TX', 'Midland'), ('DC', 'Washington'), ('OR', 'Portland'), ('CA', 'Sacramento'), ('NY', 'New York'),
            ('MI', 'Royal Oak'), ('AR', 'Bentonville'), ('CA', 'Costa Mesa'), ('CA', 'San Marcos'), ('MI', 'Ann Arbor'),
            ('NJ', 'Jersey City'), ('MA', 'Somerville'), ('CO', 'Thornton'), ('CA', 'Vista'), ('CA', 'Long Beach'),
            ('PA', 'Pittsburgh'), ('MA', 'Quincy'), ('CA', 'Napa'), ('OR', 'Hillsboro'), ('CO', 'Denver'),
            ('CA', 'Hayward')]

In [15]:
data_aggregate = pd.DataFrame()

for i in range(0, len(gent)):
    single_city_df: pd.DataFrame = gentrified.xs([gent[i][0], gent[i][1]], level=['State', 'City'])
    single_city_df.fillna(method='bfill', inplace=True)

    if single_city_df.isna().values.any() == False:
        data_aggregate = data_aggregate.append(single_city_df, ignore_index=False)
    else: 
        print(gent[i][0], gent[i][1])
data_aggregate = data_aggregate.resample('A').mean()
data_aggregate.set_index(data_aggregate.index.year, inplace=True)
data_aggregate.reset_index(inplace=True)
data_aggregate.insert(0, "State", 'GENT_AVG')
data_aggregate.insert(1, 'City', 'GENT_AVG')    

/anaconda/envs/CIS600/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


SC Charleston
TX Midland


In [16]:
data_aggregate

,State,City,Date,PriceToRentRatio_AllHomes,ZRI_AllHomes,ZriPerSqft_AllHomes,Zri_SingleFamilyResidenceRental
0,GENT_AVG,GENT_AVG,2010,14.120115,1687.160920,1.236736,1731.850575
1,GENT_AVG,GENT_AVG,2011,13.477414,1717.321839,1.261351,1763.370690
2,GENT_AVG,GENT_AVG,2012,13.493534,1750.606322,1.290908,1793.359195
3,GENT_AVG,GENT_AVG,2013,14.707213,1820.551724,1.342598,1859.735632
4,GENT_AVG,GENT_AVG,2014,15.361609,1930.813218,1.414253,1967.186782
5,GENT_AVG,GENT_AVG,2015,15.245862,2090.109195,1.539569,2135.922414
6,GENT_AVG,GENT_AVG,2016,15.726494,2203.390805,1.629098,2245.747126
7,GENT_AVG,GENT_AVG,2017,16.647184,2266.129310,1.679609,2303.793103
